In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import gc
from mpl_toolkits.mplot3d import Axes3D
import os
import psutil
process = psutil.Process()

import matplotlib.style as mplstyle
mplstyle.use('fast')

In [2]:
activation_cache_dir = f'../cache/llama/wikitext2_128/per_sample'

batch_index = 1
activation_cache_file = f'{activation_cache_dir}/{batch_index}.cache'

layer_inps: torch.Tensor
if os.path.exists(activation_cache_file):
    layer_inps = torch.load(activation_cache_file, map_location='cpu')
else:
    layer_inps = []
    for i in range(33):
        print(f'Loading layer {i}... {process.memory_info().rss / 1024 ** 2:.2f} MB')  # in bytes
        inps: torch.Tensor = torch.load(f'../cache/llama/wikitext2_128/inps_{i}.cache',map_location='cpu')
        layer_inps.append(inps[batch_index].detach().clone())
        del inps
        gc.collect()
    layer_inps = torch.stack(layer_inps) # [33, 2048, 4096]
    torch.save(layer_inps, activation_cache_file)

In [3]:
magnitudes = layer_inps.square().mean(dim=2) # [33, 2048]

# why inf?
pd.DataFrame(magnitudes)

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.000062,0.000053,0.000280,0.000175,0.000062,0.000053,0.000273,0.000204,0.000092,0.000207,...,0.000294,0.000192,0.000100,0.000067,0.000079,0.000159,0.000076,0.000079,0.000136,0.000067
1,0.005394,0.003448,0.001135,0.000717,0.001913,0.002621,0.001190,0.001040,0.001451,0.001002,...,0.001126,0.000604,0.000955,0.000996,0.000845,0.000782,0.000816,0.000815,0.000710,0.001006
2,inf,0.002283,0.002605,0.001857,0.001141,0.001663,0.003233,0.002012,0.001747,0.001733,...,0.002523,0.001508,0.001378,0.001397,0.001754,0.001734,0.001574,0.001613,0.001580,0.001241
3,inf,0.003546,0.004612,0.004093,0.003283,0.002754,0.004211,0.003422,0.004364,0.003853,...,0.003523,0.004860,0.002621,0.002699,0.002712,0.003168,0.003155,0.003069,0.003031,0.002657
4,inf,0.006786,0.009163,0.009514,0.006512,0.005569,0.006641,0.006161,0.007351,0.006897,...,0.006321,0.007332,0.004436,0.004578,0.004051,0.005661,0.006401,0.005638,0.005814,0.005375
5,inf,0.013329,0.014656,0.013893,0.011482,0.011642,0.011078,0.012695,0.011337,0.010117,...,0.011353,0.009888,0.007912,0.009468,0.006516,0.010414,0.012833,0.012978,0.011528,0.008911
6,inf,0.028122,0.022217,0.021072,0.021179,0.019928,0.015717,0.020264,0.018494,0.014328,...,0.015686,0.012924,0.013176,0.017197,0.008904,0.015320,0.016434,0.017227,0.017227,0.014297
7,inf,0.038086,0.032715,0.030670,0.028320,0.025482,0.021957,0.025711,0.024704,0.028397,...,0.020966,0.017471,0.020432,0.018600,0.012550,0.022507,0.020752,0.022781,0.020615,0.017075
8,inf,0.042419,0.042206,0.042847,0.036987,0.034760,0.030807,0.032837,0.033325,0.036194,...,0.026459,0.026062,0.026169,0.022614,0.018677,0.026627,0.029541,0.029816,0.028122,0.024033
9,inf,0.049500,0.051239,0.054871,0.046997,0.048981,0.038574,0.039001,0.041809,0.051453,...,0.032532,0.043915,0.030945,0.029831,0.028366,0.036011,0.039124,0.040649,0.035492,0.030655


In [4]:
print(pd.DataFrame(layer_inps[3].abs().max(dim=1)).T.sort_values(0, ascending=False))
layer_inps[3][:,2533].abs().sort(descending=True)

                                          0             1
113      tensor(2758., dtype=torch.float16)  tensor(2533)
0     tensor(758.5000, dtype=torch.float16)  tensor(2533)
934     tensor(1.9541, dtype=torch.float16)  tensor(1512)
988     tensor(1.9277, dtype=torch.float16)  tensor(1512)
981     tensor(1.9199, dtype=torch.float16)  tensor(1512)
...                                     ...           ...
1601    tensor(0.5005, dtype=torch.float16)   tensor(575)
919     tensor(0.4995, dtype=torch.float16)   tensor(575)
1932    tensor(0.4734, dtype=torch.float16)  tensor(3209)
713     tensor(0.4592, dtype=torch.float16)  tensor(2393)
955     tensor(0.4553, dtype=torch.float16)  tensor(2158)

[2048 rows x 2 columns]


torch.return_types.sort(
values=tensor([2.7580e+03, 7.5850e+02, 3.2178e-01,  ..., 9.1553e-05, 3.0518e-05,
        3.0518e-05], dtype=torch.float16),
indices=tensor([ 113,    0,   62,  ...,  605, 1159, 1482]))

In [5]:
print(pd.DataFrame(layer_inps[3].abs().max(dim=0)).T.sort_values(0, ascending=False))
layer_inps[3][30].abs().sort(descending=True)

                                         0             1
2533    tensor(2758., dtype=torch.float16)   tensor(113)
1415    tensor(1626., dtype=torch.float16)   tensor(113)
1512     tensor(170., dtype=torch.float16)   tensor(113)
2298  tensor(94.5000, dtype=torch.float16)   tensor(113)
2168  tensor(61.2812, dtype=torch.float16)   tensor(113)
...                                    ...           ...
2552   tensor(0.1447, dtype=torch.float16)   tensor(823)
2981   tensor(0.1442, dtype=torch.float16)  tensor(1660)
2818   tensor(0.1399, dtype=torch.float16)     tensor(0)
929    tensor(0.1390, dtype=torch.float16)     tensor(2)
3289   tensor(0.1383, dtype=torch.float16)  tensor(1051)

[4096 rows x 2 columns]


torch.return_types.sort(
values=tensor([7.9443e-01, 7.5098e-01, 6.7627e-01,  ..., 7.6294e-05, 6.1035e-05,
        6.6757e-06], dtype=torch.float16),
indices=tensor([4071, 3431, 3135,  ..., 1181, 2655, 3425]))

In [18]:
def get_max_df(single_layer_inps: torch.Tensor):
    max_df = pd.DataFrame(single_layer_inps.abs().max(dim=1)).transpose()
    max_df.columns = ["max", "channel_idx"]
    max_df["max"] = max_df["max"].apply(lambda x: x.item())
    max_df["channel_idx"] = max_df["channel_idx"].apply(lambda x: x.item())
    max_df["std"] = single_layer_inps.std(dim=1)
    # for i, row in enumerate(max_df.iterrows()):
    #     channel_idx = row[1]["channel_idx"]
    #     max_df.at[row[0], "value"] = single_layer_inps[i][channel_idx].item()
    #     max_df.at[row[0], "std"] = single_layer_inps[i].std().item()

    max_df = max_df.sort_values("max", ascending=False)
    # max_df = max_df.drop_duplicates(subset=["channel_idx"])
    return max_df

def get_magnitude_df(single_layer_inps: torch.Tensor):
    mag_df = pd.DataFrame({
        "mean_abs": single_layer_inps.abs().mean(dim=1),
        "std": single_layer_inps.std(dim=1),
    })
    # for i, row in enumerate(mag_df.iterrows()):
    #     mag_df.at[row[0], "std"] = single_layer_inps[i].std().item()
    mag_df = mag_df.sort_values("mean_abs", ascending=False)
    return mag_df

for i in range(33):
    max_df = get_max_df(layer_inps[i])
    print(f"Layer {i} max df", max_df, sep='\n')
    
    mag_df = get_magnitude_df(layer_inps[i])
    print(f"Layer {i} magnitude df", mag_df, sep='\n')

Layer 0 max df
           max  channel_idx       std
1403  0.128906         2459  0.008438
1654  0.128906         2459  0.008438
1292  0.128906         2459  0.008438
27    0.128906         2459  0.008438
210   0.128906         2459  0.008438
...        ...          ...       ...
1579  0.048828          239  0.014435
636   0.048828          239  0.014435
1601  0.048584          250  0.013153
682   0.047119         3796  0.012413
607   0.047119         3796  0.012413

[2048 rows x 3 columns]
Layer 0 magnitude df
      mean_abs       std
1114  0.015358  0.019196
1507  0.015358  0.019196
281   0.015244  0.019165
2015  0.015137  0.018967
1342  0.014992  0.018738
...        ...       ...
1562  0.001043  0.003126
1899  0.001043  0.003126
750   0.001043  0.003126
751   0.001043  0.003126
1024  0.001043  0.003126

[2048 rows x 2 columns]
Layer 1 max df
           max  channel_idx       std
0     2.992188         2298  0.073425
1     2.660156         2298  0.058716
5     2.361328         2298  

In [26]:
def find_outlier_channels(activation, threshold=20):
    mean = activation.mean(dim=(0, 1))
    std = activation.std(dim=(0, 1))
    z_scores = (activation - mean) / std
    outliers = torch.where(z_scores > threshold)
    unique_channels = torch.unique(outliers[2])
    return unique_channels.tolist()
print(find_outlier_channels(layer_inps))

[392, 575, 580, 1536, 1865, 2215, 2281, 2350, 2533, 2944, 2958, 3135, 3431, 3444, 3684, 3994, 4076]


In [24]:
# standardize the activations
layer_inps_std = (layer_inps - layer_inps.mean(dim=(0, 1), keepdim=True)) / layer_inps.std(dim=(0, 1), keepdim=True)

for i in range(33):
    max_df = get_max_df(layer_inps_std[i])
    print(f"Layer {i} max df", max_df, sep='\n')
    
    # mag_df = get_magnitude_df(layer_inps_std[i])
    # print(f"Layer {i} magnitude df", mag_df, sep='\n')

Layer 0 max df
           max  channel_idx       std
623   1.016602         1076  0.166992
978   1.015625         1076  0.166626
225   1.015625         1076  0.166626
679   1.015625         1076  0.167358
1525  1.014648         1076  0.166992
...        ...          ...       ...
340   1.000977         1076  0.166626
915   1.000977         1076  0.166382
886   1.000977         1076  0.166504
1422  1.000977         1076  0.166992
422   1.000000         1076  0.167358

[2048 rows x 3 columns]
Layer 1 max df
           max  channel_idx       std
1211  1.061523         1076  0.168823
1930  1.053711         1076  0.168457
1932  1.050781         1076  0.168213
1921  1.048828         1076  0.168213
1726  1.048828         1076  0.167603
...        ...          ...       ...
125   0.959473         1076  0.166870
126   0.958496         1076  0.166748
1172  0.958008         1076  0.166382
1167  0.957031         1076  0.166016
127   0.956543         1076  0.166748

[2048 rows x 3 columns]
Layer 2 

In [22]:
print(layer_inps_std[3][:,2533].abs().sort(descending=True))
print(layer_inps_std[3][:,1415].abs().sort(descending=True))

torch.return_types.sort(
values=tensor([4.6281e+01, 1.2688e+01, 5.0415e-02,  ..., 4.3854e-02, 4.3854e-02,
        4.2572e-02], dtype=torch.float16),
indices=tensor([ 113,    0, 1846,  ...,    1,  384,   62]))
torch.return_types.sort(
values=tensor([4.6219e+01, 1.2828e+01, 3.8727e-02,  ..., 3.0609e-02, 3.0304e-02,
        3.0106e-02], dtype=torch.float16),
indices=tensor([ 113,    0,    1,  ...,  772,  497, 2034]))


In [28]:
for i in range(33):
    print(layer_inps[i].mean(dim=0).abs().sort(descending=True))


torch.return_types.sort(
values=tensor([5.7434e-02, 5.5267e-02, 5.0446e-02,  ..., 1.2517e-06, 1.1325e-06,
        8.3447e-07], dtype=torch.float16),
indices=tensor([3964, 1411, 4030,  ..., 3160, 1459, 3460]))
torch.return_types.sort(
values=tensor([9.3262e-01, 7.1533e-01, 1.4600e-01,  ..., 1.3709e-06, 7.1526e-07,
        7.1526e-07], dtype=torch.float16),
indices=tensor([1512, 2298, 2944,  ..., 3915,  151, 3342]))
torch.return_types.sort(
values=tensor([1.6924e+00, 9.9658e-01, 6.4941e-01,  ..., 6.2585e-06, 5.3644e-06,
        2.8610e-06], dtype=torch.float16),
indices=tensor([2533, 1415, 1512,  ...,  766, 2833,  961]))
torch.return_types.sort(
values=tensor([1.7285e+00, 1.0098e+00, 8.1445e-01,  ..., 2.4855e-05, 1.8597e-05,
        1.2159e-05], dtype=torch.float16),
indices=tensor([2533, 1415, 1512,  ...,  932, 1138, 1822]))
torch.return_types.sort(
values=tensor([1.6836e+00, 1.2686e+00, 9.9219e-01,  ..., 4.2319e-06, 2.5630e-06,
        1.2517e-06], dtype=torch.float16),
indices=tensor(